In [169]:
from langchain_openai import AzureChatOpenAI
from dotenv import load_dotenv
import os
import time
from datetime import datetime
from sqlalchemy import create_engine

import pandas as pd
import warnings
warnings.filterwarnings("ignore")
load_dotenv(override=True)

True

In [170]:
required_vars = {
    "AZURE_OPENAI_ENDPOINT": os.environ.get("AZURE_OPENAI_ENDPOINT"),
    "AZURE_OPENAI_4o_DEPLOYMENT_NAME": os.environ.get("AZURE_OPENAI_4o_DEPLOYMENT_NAME"),
    "AZURE_OPENAI_API_VERSION": os.environ.get("AZURE_OPENAI_API_VERSION"),
    "AZURE_OPENAI_API_KEY": os.environ.get("AZURE_OPENAI_API_KEY"),
    "SNOWFLAKE_USER": os.environ.get("SNOWFLAKE_USER"),
    "SNOWFLAKE_PASSWORD": os.environ.get("SNOWFLAKE_PASSWORD"),
    "SNOWFLAKE_ACCOUNT": os.environ.get("SNOWFLAKE_ACCOUNT"),
    "SNOWFLAKE_WAREHOUSE": os.environ.get("SNOWFLAKE_WAREHOUSE"),
    "SNOWFLAKE_DATABASE": os.environ.get("SNOWFLAKE_DATABASE"),
    "SNOWFLAKE_SCHEMA": os.environ.get("SNOWFLAKE_SCHEMA")
}

print(required_vars["SNOWFLAKE_SCHEMA"])

TEST2


In [171]:
connection_string = (
    f"snowflake://{required_vars['SNOWFLAKE_USER']}:"
    f"{required_vars['SNOWFLAKE_PASSWORD']}@"
    f"{required_vars['SNOWFLAKE_ACCOUNT']}/"
    f"{required_vars['SNOWFLAKE_DATABASE']}/"
    f"{required_vars['SNOWFLAKE_SCHEMA']}?warehouse="
    f"{required_vars['SNOWFLAKE_WAREHOUSE']}"
)

engine = create_engine(connection_string)
print("Connected to Snowflake")

Connected to Snowflake


In [172]:
# query = f"""
#     SELECT 
#         c.TABLE_NAME, c.COLUMN_NAME, c.DATA_TYPE, c.IS_NULLABLE, c.CHARACTER_MAXIMUM_LENGTH
#     FROM {required_vars['SNOWFLAKE_DATABASE']}.INFORMATION_SCHEMA.COLUMNS c
#     JOIN {required_vars['SNOWFLAKE_DATABASE']}.INFORMATION_SCHEMA.TABLES t 
#         ON c.TABLE_NAME = t.TABLE_NAME
#     WHERE t.TABLE_TYPE = 'BASE TABLE' 
#     AND c.TABLE_SCHEMA = '{required_vars['SNOWFLAKE_SCHEMA']}'
# """

# conn = engine.connect()
# metadata = pd.read_sql(query, conn.connection)
# metadata.columns = [col.lower() for col in metadata.columns]

# print("\nAvailable tables:", metadata['table_name'].unique())
# print(metadata.to_string)

In [173]:
query = f"""
    SELECT DISTINCT
        c.TABLE_NAME, c.COLUMN_NAME, c.DATA_TYPE, c.IS_NULLABLE, c.CHARACTER_MAXIMUM_LENGTH
    FROM {required_vars['SNOWFLAKE_DATABASE']}.INFORMATION_SCHEMA.COLUMNS c
    WHERE c.TABLE_SCHEMA = '{required_vars['SNOWFLAKE_SCHEMA']}'
"""

conn = engine.connect()
metadata = pd.read_sql(query, conn.connection)
metadata.columns = [col.lower() for col in metadata.columns]

print("\nAvailable tables:", metadata['table_name'].unique())
print(metadata.to_string())



Available tables: ['PATIENT_ADMISSIONS']
            table_name         column_name data_type is_nullable  character_maximum_length
0   PATIENT_ADMISSIONS          BLOOD_TYPE      TEXT         YES                       5.0
1   PATIENT_ADMISSIONS              DOCTOR      TEXT         YES                     100.0
2   PATIENT_ADMISSIONS              GENDER      TEXT         YES                      10.0
3   PATIENT_ADMISSIONS      ADMISSION_TYPE      TEXT         YES                      50.0
4   PATIENT_ADMISSIONS            HOSPITAL      TEXT         YES                     100.0
5   PATIENT_ADMISSIONS         ROOM_NUMBER    NUMBER         YES                       NaN
6   PATIENT_ADMISSIONS                NAME      TEXT          NO                     100.0
7   PATIENT_ADMISSIONS          MEDICATION      TEXT         YES                     255.0
8   PATIENT_ADMISSIONS  INSURANCE_PROVIDER      TEXT         YES                     100.0
9   PATIENT_ADMISSIONS      BILLING_AMOUNT    NU

In [174]:
table_name = metadata['table_name'].unique()[0]  # Get first table only

print(f"\nRetrieving data from table: {table_name}")

query = f"SELECT * FROM {required_vars['SNOWFLAKE_DATABASE']}.{required_vars['SNOWFLAKE_SCHEMA']}.{table_name}"
conn = engine.connect()
df = pd.read_sql(query, conn.connection)
print(f"Retrieved {len(df)} rows")

df.head(2)


Retrieving data from table: PATIENT_ADMISSIONS
Retrieved 55514 rows


,NAME,AGE,GENDER,BLOOD_TYPE,MEDICAL_CONDITION,DATE_OF_ADMISSION,DOCTOR,HOSPITAL,INSURANCE_PROVIDER,BILLING_AMOUNT,ROOM_NUMBER,ADMISSION_TYPE,DISCHARGE_DATE,MEDICATION,TEST_RESULTS
0,test_5@@@###,55,M,O+,Diabetes,2025-02-21,Dr. Sm1th_!!,C!ty H0sp!tal,HealthCare Inc.,2000.5,305,!!!@##ER_ADMISSION###,2025-02-27,Metformin,Stable
1,test_6_error_case,60,F,B-,Chronic pain1234,2025-02-22,Dr. John Doe,City Hospital,No Coverage!!,1800.0,401,0utp@tient123,2025-02-28,Ibuprofen-500MG!,Unkn0wn??


In [175]:
model = AzureChatOpenAI(
    azure_endpoint=required_vars["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=required_vars["AZURE_OPENAI_4o_DEPLOYMENT_NAME"],
    openai_api_version=required_vars["AZURE_OPENAI_API_VERSION"],
    openai_api_key=required_vars["AZURE_OPENAI_API_KEY"],
)

In [176]:
metadata['table_name'].unique()

array(['PATIENT_ADMISSIONS'], dtype=object)

In [177]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit

# Assuming df is your DataFrame with potentially millions or billions of rows

# Check if 'target' column exists, if not, create a dummy one
if 'target' not in df.columns:
    print("No 'target' column found. Creating a dummy one for demonstration purposes.")
    df['target'] = np.random.choice(['A', 'B', 'C'], size=len(df), p=[0.4, 0.3, 0.3])

# Map target to numerical values if necessary
target_map = {'A': 0, 'B': 1, 'C': 2}  # Example mapping
df['target_num'] = df['target'].map(target_map)

# Define features (X) and target (y)
X = df.drop(['target', 'target_num'], axis=1)
y = df['target_num']

# Determine the target sample size range
target_min_sample_size = 1000
target_max_sample_size = 1300

# Calculate the fraction needed for sampling based on the total number of rows
fraction = min(target_max_sample_size / len(df), 1.0)  # Ensure fraction does not exceed 1.0

# Perform stratified sampling
sss = StratifiedShuffleSplit(n_splits=5, test_size=fraction, random_state=42)

best_sample = None
best_criteria = df['target'].value_counts().std()  # Initialize with initial dataset criteria

for train_index, test_index in sss.split(X, y):
    sampled_indices = df.index[test_index]
    sampled_df = df.loc[sampled_indices].copy()
    
    # Calculate your criteria here (e.g., class balance)
    criteria = sampled_df['target'].value_counts().std()  # Example criteria
    
    # Check if the sample size is within the desired range
    sample_size = len(sampled_df)
    if sample_size >= target_min_sample_size and sample_size <= target_max_sample_size:
        if best_criteria is None or criteria < best_criteria:
            best_sample = sampled_df
            best_criteria = criteria

print("Best Sample:")
print(best_sample)


No 'target' column found. Creating a dummy one for demonstration purposes.
Best Sample:
                    NAME  AGE  GENDER BLOOD_TYPE MEDICAL_CONDITION  \
32429     stePHANIe DUke   68    Male         O-            Asthma   
12559  kIMberLy GUERreRo   71    Male        AB-      Hypertension   
14196     sanDRA dOuGlAS   27    Male         A-           Obesity   
28189      BranDy tuRner   45    Male        AB-            Asthma   
38852    liSa RiChARdSON   45    Male        AB-          Diabetes   
...                  ...  ...     ...        ...               ...   
34697   meLIsSa cAMpbElL   20  Female         O+      Hypertension   
33784       keLLY gArNER   39    Male        AB-            Asthma   
9011    ElIZAbeTh GUZMAN   67  Female         B-          Diabetes   
3386    kRistINE mOnTOYa   60    Male         O+          Diabetes   
53500        chad mURrAY   35    Male        AB-            Asthma   

      DATE_OF_ADMISSION                DOCTOR                     HOSPI

In [178]:
len(best_sample)

1300

In [179]:
best_sample['BLOOD_TYPE'].unique()

array(['O-', 'AB-', 'A-', 'B+', 'B-', 'AB+', 'A+', 'O+'], dtype=object)

##  this prompt gives all bad data issues in given sample

In [180]:
# analysis_results = []


# data_issues_prompt = f"""
# Analyze this data sample ({len(final_sample)} total rows):

# Table Data:
# {final_sample.to_string()}

# Table Metadata:
# {metadata[['column_name', 'data_type']].to_string()}

# Please provide a comprehensive analysis focusing on:
# 1. Data quality issues
# 2. Pattern anomalies
# 3. Potential sensitive data fields
# 4. Suggested improvements

# Format your response as JSON with these keys:
# - data_quality_issues
# - recommended_solutions
# - sql_queries
# - sensitive_data_recommendations
# """

# system_prompt_quality = """You are a specialized data analyst expert in Snowflake databases.
# Analyze the provided data sample focusing on data quality and patterns.
# Keep responses focused and brief. Ensure JSON format."""

# messages_quality = [
#     {"role": "system", "content": system_prompt_quality},
#     {"role": "user", "content": data_issues_prompt}
# ]

# print("Generating data quality insights...")
# quality_response = model.invoke(messages_quality).content.replace("plaintext", "").replace("json", "").replace("```", "").strip()
# print("\nData quality insights generated")

# analysis_results = [{
# 'chunk': 'Representative Sample',
# 'analysis': quality_response
#     }]

# # Print results
# for result in analysis_results:
#     print(result['analysis'])


In [191]:
print(metadata[['column_name']].to_string())

           column_name
0           BLOOD_TYPE
1               DOCTOR
2               GENDER
3       ADMISSION_TYPE
4             HOSPITAL
5          ROOM_NUMBER
6                 NAME
7           MEDICATION
8   INSURANCE_PROVIDER
9       BILLING_AMOUNT
10   DATE_OF_ADMISSION
11   MEDICAL_CONDITION
12                 AGE
13      DISCHARGE_DATE
14        TEST_RESULTS


In [200]:
print(metadata[['column_name', 'data_type']].to_string())

           column_name data_type
0           BLOOD_TYPE      TEXT
1               DOCTOR      TEXT
2               GENDER      TEXT
3       ADMISSION_TYPE      TEXT
4             HOSPITAL      TEXT
5          ROOM_NUMBER    NUMBER
6                 NAME      TEXT
7           MEDICATION      TEXT
8   INSURANCE_PROVIDER      TEXT
9       BILLING_AMOUNT    NUMBER
10   DATE_OF_ADMISSION      DATE
11   MEDICAL_CONDITION      TEXT
12                 AGE    NUMBER
13      DISCHARGE_DATE      DATE
14        TEST_RESULTS      TEXT


##  this prompt gives all DQ rule suggestions for given stratified sample

In [183]:
dq_rule_prompt = f"""
# Data Quality Rule Generation Protocol

## Analysis Context
Analyze this data sample ({len(best_sample)} total rows):

Table Data:
{best_sample.to_string()}

Table Metadata:
{metadata[['column_name', 'data_type']].to_string()}

# Technical Requirements
    You are an enterprise data quality engine tasked with creating precise data quality rules based on the provided data sample. The output must:
    Proactively Identify Potential Data Quality Issues: Develop rules that can detect anomalies or inconsistencies that might arise in the future, even if they are not present in the current sample.
    Provide Technical Validations: Offer SQL-based validation expressions to address identified issues.
    Specify Compliance Concerns and Masking Techniques: Identify potential compliance risks and suggest appropriate masking techniques.
    Prioritize Rules Based on Severity and Business Impact: Assess the potential impact of each rule on business operations.
    Include SQL-Based Validation Expressions: Provide SQL queries to validate rule implementation.

##Expected JSON Output Format
 Return a JSON object with the following structure:

{{
    "data_quality_rules": [
        {{
            "rule_name": "DQR_{{TABLE}}{{RULE_TYPE}}{{##}}",
            "rule_type": "One of [NOT_NULL, UNIQUENESS, RANGE, PATTERN, REFERENTIAL, CUSTOM]",
            "affected_columns": ["column_names"],
            "validation_expression": "SQL expression or technical rule implementation",
            "severity": "One of [HIGH, MEDIUM, LOW]",
            "validation_sql": "Technical validation SQL to verify rule implementation"
        }}
    ],
    "compliance_rules": [
        {{
        "column": all of [{metadata[['column_name']].to_string()}],
        "compliance_standard": ["applicable standards like PII, PHI, PCI, HIPAA, GDPR, SOC2"],
        "masking_technique": "suggested technique",
        "severity": "One of [HIGH, MEDIUM, LOW]",
        "validation_sql": "SQL to identify compliance violations"
        }}
    ],
        "anomaly_detection_rules": [
        {{
            "description": "Technical description of potential anomaly detection logic",
            "affected_columns": ["column_names"],
            "detection_expression": "SQL or logic to detect potential anomalies",
            "severity": "One of [HIGH, MEDIUM, LOW]",
            "recommended_action": "Specific technical remediation approach"
        }}
    ]
}}

# Analysis Guidelines
    Evaluate Potential Data Quality Risks: Assess completeness, accuracy, consistency, validity, timeliness, uniqueness, and integrity to anticipate potential issues.
    Identify Industry-Specific Compliance Violations: Consider all columns from metadata for compliance rules.
    Detect Statistical Outliers and Distribution Irregularities: Develop rules to identify anomalies that might arise in the future.
    Assess Pattern Inconsistencies or Format Violations: Create rules to detect inconsistencies in data patterns or formats.
    Evaluate Semantic Data Quality Issues: Consider data governance or stewardship concerns beyond basic metrics.
    Use all columns for Compliance rules.
    Consider Potential Data Governance or Stewardship Concerns: Develop rules that align with organizational data policies.

# Technical Response Requirements
    Ensure each rule has a unique identifier following the naming convention.
    For every identified issue, provide a specific SQL validation query.
    Include severity ratings based on business impact.
    Specify clear implementation phases and complexity.
    Provide concrete technical recommendations.
    This refined prompt focuses on generating proactive data quality rules that can detect potential anomalies, even if they are not present in the current sample dataset.
"""


system_prompt_dq_rule = """You are a specialized data quality analyst expert in Snowflake databases.
Analyze the provided data sample focusing on suggesting data quality rules and patterns.
Keep responses focused and brief. Ensure JSON format."""

messages_quality = [
    {"role": "system", "content": system_prompt_dq_rule},
    {"role": "user", "content": dq_rule_prompt}
]

print("Generating data quality insights...")
dq_response = model.invoke(messages_quality).content.replace("plaintext", "").replace("json", "").replace("```", "").strip()
print("\nData quality insights generated")

dq_results = [{
'chunk': 'Representative Sample',
'analysis': dq_response
    }]

# Print results
for result in dq_results:
    print(result['analysis'])

Generating data quality insights...

Data quality insights generated
{
    "data_quality_rules": [
        {
            "rule_name": "DQR_NAME_FORMAT_01",
            "rule_type": "PATTERN",
            "affected_columns": ["NAME"],
            "validation_expression": "Ensure names are in 'Title Case' format.",
            "severity": "MEDIUM",
            "validation_sql": "SELECT * FROM table WHERE NAME != INITCAP(NAME);"
        },
        {
            "rule_name": "DQR_AGE_RANGE_02",
            "rule_type": "RANGE",
            "affected_columns": ["AGE"],
            "validation_expression": "Age should be between 0 and 120.",
            "severity": "HIGH",
            "validation_sql": "SELECT * FROM table WHERE AGE < 0 OR AGE > 120;"
        },
        {
            "rule_name": "DQR_GENDER_VALIDITY_03",
            "rule_type": "CUSTOM",
            "affected_columns": ["GENDER"],
            "validation_expression": "Gender should be one of 'Male', 'Female', or 'Other'.",

### save json output in excel

In [184]:
import pandas as pd
import json

try:
    # First, make sure we have valid JSON
    json_data = json.loads(dq_response)
    
    # Create Excel writer
    with pd.ExcelWriter('data_quality_rules.xlsx') as writer:
        # Create dataframes from each section of the JSON
        if 'data_quality_rules' in json_data:
            dq_rules_df = pd.DataFrame(json_data['data_quality_rules'])
            dq_rules_df.to_excel(writer, sheet_name='Data Quality Rules', index=False)
            
        if 'compliance_rules' in json_data:
            compliance_df = pd.DataFrame(json_data['compliance_rules'])
            compliance_df.to_excel(writer, sheet_name='Compliance Rules', index=False)
            
        if 'anomaly_detection_rules' in json_data:
            anomaly_df = pd.DataFrame(json_data['anomaly_detection_rules']) 
            anomaly_df.to_excel(writer, sheet_name='Anomaly Rules', index=False)
            
    print("JSON data successfully saved to 'data_quality_rules.xlsx'")
    
except json.JSONDecodeError as e:
    print(f"Error parsing JSON response: {e}")
    print("Original response:")
    print(dq_response)

JSON data successfully saved to 'data_quality_rules.xlsx'
